In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder
from io import StringIO


In [8]:
with open("../data/data1.txt", "r") as file:
    content = file.read()

cleaned_content = "\n".join([line.rstrip(",") for line in content.splitlines()])

df1 = pd.read_csv(StringIO(cleaned_content), sep=",")
df1




,pdv,fecha,pago,cp,banco
0,PDV 91861,20211004,40.00,1,BANCO 9
1,PDV 98472,20211210,53.00,1,BANCO 9
2,PDV 94996,20210821,18.10,1,BANCO 9
3,PDV 49370,20220429,100.00,1,BANCO 9
4,PDV 111466,20230203,100.00,1,BANCO 9
...,...,...,...,...,...
995,PDV 107176,20230508,100.00,1,BANCO 85
996,PDV 105893,20220223,20.00,1,BANCO 9
997,PDV 101788,20220520,60.00,1,BANCO 82
998,PDV 106451,20220326,20.00,1,BANCO 9


In [10]:

with open("../data/data2.txt", "r") as file:
    content = file.read()

cleaned_content = "\n".join([line.rstrip(",") for line in content.splitlines()])

df2 = pd.read_csv(StringIO(cleaned_content), sep=",")
df2



,ventas,cantidad,pdv,producto,fecha,categoria
14,0,8,PDV 104074,PRODUCTO 9,20210727,RECARGAS
20,0,4,PDV 15896,PRODUCTO 27,20230524,RECARGAS
6,0,3,PDV 21017,PRODUCTO 27,20230706,RECARGAS
98,84,6,PDV 50196,PRODUCTO 44,20210724,RECAUDACION
6,0,6,PDV 111013,PRODUCTO 9,20221109,RECARGAS
...,...,...,...,...,...,...
1,0,1,PDV 39706,PRODUCTO 83,20210703,RECARGAS
9,0,3,PDV 103118,PRODUCTO 83,20220825,RECARGAS
4,0,1,PDV 47638,PRODUCTO 1,20210727,RECARGAS
8,0,2,PDV 50276,PRODUCTO 6,20220503,RECARGAS


In [11]:
# Cargar 3.txt
with open("../data/data3.txt", "r") as file:
    content = file.read()

cleaned_content = "\n".join([line.rstrip(",") for line in content.splitlines()])

df3 = pd.read_csv(StringIO(cleaned_content), sep=",")
print(df3)

            pdv  idregion  idciudad    canal actividad
0     PDV 47873         2      1209  CANAL 3         F
1     PDV 47179         3      1409  CANAL 5         F
2     PDV 43609         3      1411  CANAL 3         F
3     PDV 47128         2        24  CANAL 3         F
4     PDV 47130         3      1409  CANAL 3         F
..          ...       ...       ...      ...       ...
995  PDV 110692         2        27  CANAL 3         F
996  PDV 110804         3      1409  CANAL 3         F
997  PDV 110887         3      1409  CANAL 3         F
998   PDV 49151         2        27  CANAL 3         F
999   PDV 49348         3      1409  CANAL 3         F

[1000 rows x 5 columns]
